In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/make data count kaggle/make-data-count-finding-data-references (1)/train_labels.csv")

In [3]:
data

,article_id,dataset_id,type
0,10.1002_2017jc013030,https://doi.org/10.17882/49388,Primary
1,10.1002_anie.201916483,https://doi.org/10.5517/ccdc.csd.cc1npvt0,Missing
2,10.1002_anie.202005531,https://doi.org/10.5517/ccdc.csd.cc24wxqp,Missing
3,10.1002_anie.202007717,https://doi.org/10.5517/ccdc.csd.cc24rrb0,Missing
4,10.1002_chem.201902131,https://doi.org/10.5517/ccdc.csd.cc221dk3,Missing
...,...,...,...
1061,10.7717_peerj.12422,https://doi.org/10.15468/dl.t3h8b4,Secondary
1062,10.7717_peerj.12422,https://doi.org/10.15468/dl.yak5vd,Secondary
1063,10.7717_peerj.13193,https://doi.org/10.6073/pasta/02e2764efb408a8b...,Secondary
1064,10.7717_peerj.13193,https://doi.org/10.6073/pasta/275ad28a2f31356c...,Secondary


In [4]:
from pathlib import Path
xml_dir = Path("/content/drive/MyDrive/make data count kaggle/make-data-count-finding-data-references (1)/train/XML")
pdf_dir = Path("/content/drive/MyDrive/make data count kaggle/make-data-count-finding-data-references (1)/train/PDF")
labels_path = Path("/content/drive/MyDrive/make data count kaggle/make-data-count-finding-data-references (1)/train_labels.csv")

In [5]:
labels_df = pd.read_csv(labels_path)
label_ids = set(labels_df['article_id'].unique())

In [3]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 92.4 MB/s eta 0:00:00


In [4]:
!pip install helpers

In [ ]:
# Install dependencies first
!pip install pymupdf beautifulsoup4 lxml
from google.colab import drive
drive.mount('/content/drive')

# Your optimized code with minor enhancements:
import re
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from pathlib import Path
import logging
import pymupdf

# Enhanced logger setup
l = logging.getLogger("pdf_xml_parser")
if not l.hasHandlers():
    l.setLevel(logging.INFO)  # Use INFO for less verbose output
    ch = logging.StreamHandler()
    formatter = logging.Formatter('%(levelname)s - %(message)s')
    ch.setFormatter(formatter)
    l.addHandler(ch)

# Directories (using pathlib consistently)
base = Path("/content/drive/MyDrive/make data count kaggle/make-data-count-finding-data-references (1)")
xml_dir = base / "train/XML"
pdf_dir = base / "train/PDF"
output_dir = base / "parsed_output_part_2"
output_dir.mkdir(parents=True, exist_ok=True)

def is_header_footer(text: str, page_num: int, total_pages: int) -> bool:
    patterns = [
        r"^\s*\d{1,3}\s*$",
        r"^.*\|\s*\d{1,3}\s*of\s*\d{1,3}\s*\|.*$",
        r"^.*\d{1,2}\s*[–-]\s*\d{1,2}\s*[a-zA-Z]+\s*\d{4}.*$",
        r"^.*©.*$",
        r"^.*confidential.*$",
        r"^.*draft.*$",
        r"^.*(this\s+copy\s+is\s+for\s+review).*$",
    ]
    for pattern in patterns:
        if re.match(pattern, text, re.IGNORECASE):
            return True
    if page_num == 1 and ("abstract" in text.lower() or "introduction" in text.lower()):
        return False
    if page_num == total_pages and ("references" in text.lower() or "acknowledg" in text.lower()):
        return False
    return False

def extract_text_from_pdf(pdf_file: Path) -> str:
    full_text = []
    try:
        with pymupdf.open(pdf_file) as doc:
            total_pages = len(doc)
            for page_num, page in enumerate(doc, start=1):
                blocks = page.get_text("blocks", sort=True)
                page_text = [
                    re.sub(r'\s+', ' ', block[4]).strip()
                    for block in blocks
                    if block[6] == 0 and not is_header_footer(block[4], page_num, total_pages)
                ]
                if page_text:
                    full_text.append("\n".join(page_text))
        return "\n\n".join(full_text)
    except Exception as e:
        l.warning(f"PDF error: {pdf_file.name} - {str(e)}")
        return ""

def extract_text_from_xml(xml_file: Path) -> str:
    try:
        with open(xml_file, 'r', encoding='utf-8', errors='replace') as f:
            content = f.read()

        # Parse with BeautifulSoup first
        try:
            soup = BeautifulSoup(content, 'lxml-xml')
            for tag in soup(['ref-list', 'bibliography', 'table-wrap', 'fig']):
                tag.decompose()
            text = soup.get_text(separator='\n', strip=True)
        except:
            text = ""

        # Fallback to ElementTree
        if not text:
            try:
                root = ET.fromstring(content)
                text = "\n".join([
                    el.text.strip() for el in root.findall('.//')
                    if el.text and el.text.strip()
                ])
            except:
                text = re.sub(r'<[^>]+>', '', content)  # Raw tag removal

        # Post-processing
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\n{3,}', '\n\n', text)
        text = re.sub(r'\b(doi|DOI):\s*(\S+)', r'https://doi.org/\2', text)  # Enhanced DOI handling
        return text.strip()
    except Exception as e:
        l.warning(f"XML error: {xml_file.name} - {str(e)}")
        return ""

# is_header_footer remains identical to your version

def process_file(file_path: Path):
    txt_file = output_dir / f"{file_path.stem}.txt"
    if txt_file.exists():
        return  # Skip existing

    processor = {
        '.pdf': extract_text_from_pdf,
        '.xml': extract_text_from_xml,
        '.nxml': extract_text_from_xml
    }.get(file_path.suffix.lower())

    if not processor:
        l.warning(f"Skipped unsupported file: {file_path.name}")
        return

    if text := processor(file_path):
        txt_file.write_text(text, encoding='utf-8')
        l.info(f"Created: {txt_file.name} ({len(text)//1000}KB)")
    else:
        txt_file.write_text("", encoding='utf-8')
        l.warning(f"Empty output: {file_path.name}")

# Main processing
l.info("STARTING PROCESSING")
file_extensions = ['.pdf', '.xml', '.nxml']
file_paths = [
    f for f in [*pdf_dir.iterdir(), *xml_dir.iterdir()]
    if f.suffix.lower() in file_extensions
]
l.info(f"Found {len(file_paths)} files to process")

for path in file_paths:
    process_file(path)

l.info(f"COMPLETED! Output in {output_dir}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appea

In [ ]:
import pandas as pd
from pathlib import Path

# Set up paths
parsed_dir = Path("/content/drive/MyDrive/make data count kaggle/make-data-count-finding-data-references (1)/parsed_output_part_2")
labels_path = Path("/content/drive/MyDrive/make data count kaggle/make-data-count-finding-data-references (1)/train_labels.csv")

# Read labels CSV
labels_df = pd.read_csv(labels_path)

# Create a mapping from article_id to text content
text_mapping = {}
for txt_file in parsed_dir.glob("*.txt"):
    article_id = txt_file.stem
    text_content = txt_file.read_text(encoding="utf-8")
    text_mapping[article_id] = text_content

# Map labels to text content
def map_text_to_labels(row):
    article_id = row["article_id"]
    return text_mapping.get(article_id, "")

# Apply mapping and filter out missing texts
labels_df["text"] = labels_df.apply(map_text_to_labels, axis=1)
mapped_df = labels_df[labels_df["text"] != ""].reset_index(drop=True)

print(f"Mapped {len(mapped_df)}/{len(labels_df)} records")
print("Sample mapped records:")
print(mapped_df.head(2))

mapped_df.to_csv("/content/drive/MyDrive/make data count kaggle/make-data-count-finding-data-references (1)/mapped_data.csv", index=False)